In [110]:
import requests
import json
import pprint

In [106]:
url = "http://lims.dccouncil.us/api/v1/Legislation/advancedSearch"

In [134]:
head = {'content-type':'application/json','User-agent': 'Mozilla/5.0'} 
q = {"CouncilPeriod": "23"}
payload = {'rowLimit':'5'}


In [135]:
response = requests.post(url,json=q ,headers=head,params=payload)

In [138]:
pp = pprint.PrettyPrinter(indent=7)
d = response.text
text = json.loads(d)
a = pp.pprint(text[1])

{      'ActNumber': None,
       'Category': 'Bill',
       'CommitteeReferral': 'Committee on Transportation and the Environment '
                            'and Committee of the Whole',
       'CommitteeReferralComments': '',
       'CouncilPeriod': 23,
       'Deemed': None,
       'DeemedOn': None,
       'DocumentUrl': 'http://lims.dccouncil.us/Download/42098/B23-0193-Introduction.pdf',
       'Id': 42098,
       'IntroductionDate': '2019-03-19',
       'LawNumber': None,
       'LegislationNumber': 'B23-0193',
       'LegislationStatus': 'New',
       'Modified': '2019-03-20',
       'ShortTitle': 'Electric Vehicle Readiness Amendment Act of 2019',
       'Subcategory': 'Permanent Bill'}


In [45]:
response.url

'http://lims.dccouncil.us/api/v1/Legislation/Search?rowLimit=5'

In [140]:
url_details = "http://lims.dccouncil.us/api/v1/Legislation/Details"
head = {'content-type':'application/json','User-agent': 'Mozilla/5.0'} 
payload = {'legislationNumber':'B23-0193'}

In [99]:
url

'http://lims.dccouncil.us/api/v1/Legislation/Details'

In [141]:
response = requests.get(url_details, headers=head, params=payload)

In [101]:
response.url

'http://lims.dccouncil.us/api/v1/Legislation/Details?legislationNumber=PR19-0045'

In [169]:
d = response.text
text = json.loads(d)
a = pp.pprint(text)

{      'CommitteeMarkup': [],
       'CongressReview': [],
       'CouncilReview': [],
       'Hearing': [],
       'Legislation': {      'ActNumber': None,
                             'AdditionalInformation': None,
                             'AgendaDate': None,
                             'Category': 'Bill',
                             'CoSponsor': 'Councilmembers Evans, Bonds, Todd, '
                                          'Allen, Grosso, R. White, and '
                                          'Chairman Mendelson',
                             'CommitteeReferral': 'Committee on Transportation '
                                                  'and the Environment and '
                                                  'Committee of the Whole',
                             'CommitteeReferralComments': '',
                             'CouncilPeriod': 23,
                             'DateRead': '2019-03-19',
                             'Deemed': None,
                     

In [150]:
text['Legislation']['LegislationStatus']

'New'

In [163]:
import copy
text2 = copy.deepcopy(text)

In [164]:
text2['Legislation']['LegislationStatus'] = "Pending"

In [165]:
text2 == text

False

In [180]:
text.keys()

dict_keys(['LinkedLegislations', 'VotingSummary', 'MayorReview', 'OtherDocuments', 'Hearing', 'CouncilReview', 'CommitteeMarkup', 'CongressReview', 'Legislation'])

In [181]:
for key in text.keys():
    same = text2[key] == text[key]
    print("{} key is: {}".format(key,same))

LinkedLegislations key is: True
VotingSummary key is: True
MayorReview key is: True
OtherDocuments key is: True
Hearing key is: True
CouncilReview key is: True
CommitteeMarkup key is: True
CongressReview key is: True
Legislation key is: False


In [205]:
html_str = "<h1>The legislation status for {} is</h1><table>".format("BS-12412")
for key in text.keys():
    print(html_str)
    tp = text[key]
    if isinstance(tp,dict):
        html_str += "<li> Legislation: <ul>"
        for key_2 in text[key]:       
            html_str = html_str + ("<li> {} : {}</li>".format(key_2,text[key][key_2]))
        html_str = html_str + "</li></ul>"
    else:
        html_str = html_str + "<li>{} is {}</li>".format(key,tp)
html_str = html_str + "</ul>"

with open('output.html','w') as f:
    f.write(html_str)

dict

In [182]:
import pandas as pd

# Trying to get all the keys to build a df

In [ ]:
def pd_headers(text):
    header_0_indent = []
    header_1_indent = []

    for key in text.keys():
        tp = text[key]
        if isinstance(tp,dict):
            for key_2 in text[key]:       
                header_1_indent.append(key_2)
        else:
            header_0_indent.append(key)
    header_full = [header_0_indent,header_1_indent]
    return(header_full)



In [297]:
pd.set_option('max_colwidth',10000)
a = pd_headers(text)
df = pd.DataFrame(columns=a[1])
i = 0
added_row = []
for item in a[1]:  
    added_row.append(text['Legislation'][item])
df.loc[len(df)] = added_row
added_row = []
for item in a[1]:  
    added_row.append(text2['Legislation'][item])
    
df.loc[len(df)] = added_row

df.to_html('output.html')

In [296]:
df['CommitteeReferral']


0    Committee on Transportation and the Environment and Committee of the Whole
1    Committee on Transportation and the Environment and Committee of the Whole
Name: CommitteeReferral, dtype: object

In [254]:
def setup_table(text):
    html_str = """<style>table, th, td {
      border: 1px solid black;
    }</style>
    <h1>The legislation status for BS-12412 is</h1><table><tr>"""
    for key in text.keys():
        tp = text[key]
        if isinstance(tp,dict):
            for key_2 in text[key]:       
                html_str = html_str + ("<th>{}</th>".format(key_2))
        else:
            html_str = html_str + "<th>{}</th>".format(key)

    html_str += "</tr>"
    return(html_str)


def add_table_items(text,html_str):
    html_str += "<tr>"                                   
    for key in text.keys():

        tp = text[key]
        if isinstance(tp,dict):
            for key_2 in text[key]:       
                html_str = html_str + ("<td>{}</td>".format(text[key][key_2]))
        else:
            html_str = html_str + "<td>{}</td>".format(tp)
    return(html_str)

            
            
def finish_table(text,html_str):            
    html_str = html_str + "</tr></table>"

    with open('output.html','w') as f:
        f.write(html_str)

In [260]:
html_str = setup_table(sorted(text))
html_str = add_table_items(text,html_str)
html_str = add_table_items(text2,html_str)
finish_table(text,html_str)

AttributeError: 'list' object has no attribute 'keys'

In [304]:
# This is useful for checking changes
for key in df.keys():
    if df[key][0] != df[key][1]:
        print("{} is not equal to {}".format(df[key][0],df[key][1]))


New is not equal to Pending


In [438]:
#Resets dataframe
a = pd_headers(text_details)
df = pd.DataFrame(columns=a[1])

In [439]:
#Advance search for recent
url = "http://lims.dccouncil.us/api/v1/Legislation/advancedSearch"
head = {'content-type':'application/json','User-agent': 'Mozilla/5.0'} 
q = {"CouncilPeriod": "23"}
payload = {'rowLimit':'100'}
response = requests.post(url,json=q ,headers=head,params=payload)
d = response.text
text = json.loads(d)





In [440]:
i = 0
for i in range(len(text)):
    
    leg_num = text[i]['LegislationNumber']
    #print(leg_num)
    # Individual details
    url_details = "http://lims.dccouncil.us/api/v1/Legislation/Details"
    head_details = {'content-type':'application/json','User-agent': 'Mozilla/5.0'} 
    payload_details = {'legislationNumber':leg_num}
    response_details = requests.get(url_details, headers=head_details, params=payload_details)

    d_details = response_details.text
    text_details = json.loads(d_details)
    added_row = []
    
    for item in a[1]:
        try:
            added_row.append(text_details['Legislation'][item])
        except:
            added_row.append('MISSING {}'.format(item))
    df.loc[len(df)] = added_row
    i+=1

In [441]:
df.to_html('output.html')
df.to_csv('output.csv',index=0)

#df.ix[0,'PlaceOfIntroduction'] = 'Mayor'
#df.to_csv('output_2.csv',index=0)


In [435]:
df = pd.read_csv('output.csv')
df_2 = pd.read_csv('output_2.csv')

In [411]:
text[9]['LegislationNumber']

'B23-0200'

In [437]:
for key in df.keys():
    if df[key][0] != df_2[key][0]:
        if str(df[key][0]) != 'nan':
            print("{} :: CHANGE IN '{}' :: {} is not equal to {}".format(df['LegislationNumber'][0],key,df[key][0],df_2[key][0]))

B23-0204 :: CHANGE IN 'DateRead' :: 2019-04-02 is not equal to 4/2/2019
B23-0204 :: CHANGE IN 'IntroductionDate' :: 2019-03-05 is not equal to 3/6/2019
B23-0204 :: CHANGE IN 'Modified' :: 2019-03-21 is not equal to 3/21/2019


In [378]:
type(df['WithdrawnBy'][0] )


numpy.float64

In [430]:
df.head()

,WithdrawnBy,VendorName,PlaceOfIntroduction,MemoLink,DateRead,AdditionalInformation,RereferralCommitteeComments,ReIntroducer,LawNumber,IntroductionDate,...,Introducer,ShortTitle,PlaceRead,IsRereferred,AgendaDate,Id,RereferralCommittee,PublicationDate,ReCoSponsor,DeemedOn
0,None,None,Mayor,http://lims.dccouncil.us/Download/42110/B23-0204_Memorandum.pdf,2019-04-02,None,,None,None,2019-03-05,...,Councilmember Cheh,Energy Efficiency Standards Amendment Act of 2019,Legislative Meeting,False,None,42110.0,,None,None,None
1,None,None,Committee of the Whole,http://lims.dccouncil.us/Download/42097/B23-0192_Memorandum.pdf,2019-03-19,None,,None,None,2019-03-19,...,Councilmembers Cheh and Gray,Cottage Food Expansion Amendment Act of 2019,Committee of the Whole,False,None,42097.0,,None,None,None
2,None,None,Committee of the Whole,http://lims.dccouncil.us/Download/42098/B23-0193_Memorandum.pdf,2019-03-19,None,,None,None,2019-03-19,...,Councilmember Cheh,Electric Vehicle Readiness Amendment Act of 2019,Committee of the Whole,False,None,42098.0,,None,None,None
3,None,None,Committee of the Whole,http://lims.dccouncil.us/Download/42100/B23-0195_Memorandum.pdf,2019-03-19,None,,None,None,2019-03-19,...,"Councilmembers Grosso, Cheh, Todd, Nadeau, and R. White",Michael A. Stoops Anti-Discrimination Amendment Act of 2019,Committee of the Whole,False,None,42100.0,,None,None,None
4,None,None,Committee of the Whole,http://lims.dccouncil.us/Download/42101/B23-0196_Memorandum.pdf,2019-03-19,None,,None,None,2019-03-19,...,"Councilmembers Grosso, R. White, Cheh, Gray, Allen, Silverman, Nadeau, and Bonds",District of Columbia Public Schools Student Technology Equity Act of 2019,Committee of the Whole,False,None,42101.0,,None,None,None


In [420]:
text[9]['LegislationNumber']

'B23-0200'